<a href="https://colab.research.google.com/github/aliiamrr/Crab-Age-Prediction-ML/blob/main/ML_Proj_1(Crab_Data_Set).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#DATA VISUALIZATION AND CORRELATION ANALYSIS

In [1]:
#IMPORTS
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
#READ THE DATASET
df = pd.read_csv('train.csv')

In [ ]:
#BASIC EXPLORATION
print(df.head(),'\n')
print(df.info(),'\n')
print(df.describe())

   id  Length  Diameter  Height     Weight  Shucked Weight  Viscera Weight  \
0   0  1.2875    1.0750  0.3625  19.433582        8.306404        4.309124   
1   1  1.5500    1.2125  0.4250  33.877653       13.706983        6.676307   
2   2  1.3875    1.0625  0.3500  23.388337        9.993199        5.556502   
3   3  1.4375    1.1250  0.4375  27.499015       11.254751        5.329706   
4   4  1.5250    1.2250  0.4250  32.460178       13.919605        7.569317   

   Shell Weight   Age  Sex_I  Sex_M  
0      6.662133   9.0   True  False  
1     10.064072  14.0  False  False  
2      5.811648  11.0  False   True  
3      7.937860  22.0  False  False  
4      8.788345  10.0  False   True   

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              15000 non-null  int64  
 1   Length          15000 non-null  float64
 2   Diamete

In [ ]:
#CHECK AND HANDLE DUPLICATES
'''
duplicates = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")
'''

In [ ]:
#CHECK AND HANDLE MISSING VALUES
'''
missing_values = df.isnull().sum()
print(f"Missing values:\n{missing_values}")
'''

In [4]:
#CONVERT SEX TO A NUMERICAL FEAUTURE USING ONE HOT ENCODING
df = pd.get_dummies(df, columns=['Sex'], drop_first=True)

In [6]:
#CALCULATE AND DISPLAY CORRELATION MATRIX
correlation_matrix = df.corr()
print(correlation_matrix)

                      id    Length  Diameter    Height    Weight  \
id              1.000000 -0.000549 -0.000297 -0.001960 -0.000134   
Length         -0.000549  1.000000  0.991766  0.934141  0.938050   
Diameter       -0.000297  0.991766  1.000000  0.936500  0.939962   
Height         -0.001960  0.934141  0.936500  1.000000  0.915395   
Weight         -0.000134  0.938050  0.939962  0.915395  1.000000   
Shucked Weight -0.000914  0.913607  0.912383  0.871371  0.970024   
Viscera Weight  0.002226  0.920136  0.920553  0.896615  0.972999   
Shell Weight   -0.001184  0.920366  0.925611  0.919334  0.967680   
Age            -0.009181  0.619932  0.627211  0.663039  0.610977   
Sex_I          -0.003658 -0.672572 -0.678027 -0.667699 -0.663938   
Sex_M          -0.000799  0.325454  0.326021  0.318029  0.314957   

                Shucked Weight  Viscera Weight  Shell Weight       Age  \
id                   -0.000914        0.002226     -0.001184 -0.009181   
Length                0.913607     

In [ ]:
#DETECTING OUTLIERS AND HANDLING THEM

#FUNCTION FOR OUTLIERS
def detect_outliers(data, threshold=3):
    z_scores = (data - data.mean()) / data.std()
    outliers = data[np.abs(z_scores) > threshold]
    return outliers

numerical_features = df.drop(columns=['id']).select_dtypes(include=['float64', 'int64']).columns.tolist()

for feature in numerical_features:
    outliers = detect_outliers(df[feature], threshold=3)
    print(f"Outliers in {feature}: {outliers}")

In [ ]:


for feature in numerical_features:
    stats.probplot(df[feature], plot=plt)
    plt.title(f'Q-Q Plot for {feature}')
    plt.show()

# Shapiro-Wilk Test
shapiro_test_stat, shapiro_p_value = stats.shapiro(df[numerical_features])
print(f"Shapiro-Wilk Test: Statistic={shapiro_test_stat}, p-value={shapiro_p_value}")

# Kolmogorov-Smirnov Test
ks_test_stat, ks_p_value = stats.kstest(df[numerical_features], 'norm')
print(f"Kolmogorov-Smirnov Test: Statistic={ks_test_stat}, p-value={ks_p_value}")

Method for Feature Engineering


In [ ]:
def engineer_features(df):


    new_df = pd.DataFrame()

    # Create ratio features
    new_df['Shell_Weight_Ratio'] = df['Shell Weight'] / df['Weight']
    new_df['Shucked_Weight_Ratio'] = df['Shucked Weight'] / df['Weight']
    new_df['Viscera_Weight_Ratio'] = df['Viscera Weight'] / df['Weight']

    # Create size-related features (assuming you have functions to calculate volume and surface area)
    new_df['Volume'] = calculate_volume(df['Length'], df['Diameter'], df['Height'])
    new_df['Surface_Area'] = calculate_surface_area(df['Length'], df['Diameter'], df['Height'])

    # Consider creating interaction terms if needed
    # new_df['Length_Diameter_Interaction'] = df['Length'] * df['Diameter']
    # new_df['Height_Weight_Interaction'] = df['Height'] * df['Weight']

    return new_df

In [ ]:
#STANDARDIZING NUMERICAL FEATURES
scaler = StandardScaler()
numerical_features = df.drop(columns=['id','Age']).select_dtypes(include=['float64', 'int64']).columns.tolist()
df[numerical_features] = scaler.fit_transform(df[numerical_features])

In [ ]:
#PCA(OLD)

features = df.drop(columns=['id', 'Age','Sex_M'])
pca = PCA(n_components=5)
principal_components = pca.fit_transform(features)
pca_df = pd.DataFrame(data=principal_components, columns=[f'PC{i+1}' for i in range(principal_components.shape[1])])
pca_df['Age'] = df['Age'].values
explained_variance = pca.explained_variance_ratio_
print(f"Explained variance ratio: {explained_variance}")


Explained variance ratio: [9.85764841e-01 9.48801567e-03 2.37051783e-03 1.71274259e-03
 5.86404419e-04]


In [ ]:
#PCA VISUALIZATION(OLD)

plt.figure(figsize=(10, 5))
plt.bar(range(1, len(explained_variance) + 1), explained_variance)
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance Ratio')
plt.title('Explained Variance by Principal Components')
plt.show()


We will use PC1 only since it captures 94% of the variance

In [ ]:
#VISUALIZING THE RELATIONSHIP BETWEEN PC1 AND AGE USING A SCATTER PLOT

In [ ]:

plt.figure(figsize=(10, 6))
sns.scatterplot(x='PC4', y='Age', data=pca_df, alpha=0.5)
plt.title('Scatter Plot of PC1 vs Age')
plt.xlabel('Principal Component 1 (PC1)')
plt.ylabel('Age')
plt.grid(True)
plt.show()


From the Scatter Plot it is clear that there is a relatively direct relationship between PC1 and Age.

We will now use Ridge regularization


In [ ]:
X =  features #pca_df[['PC1','PC2','PC3','PC4','PC5']]
y = pca_df['Age']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
#RIDGE

ridge_param_grid = {'alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}
ridge_grid = GridSearchCV(Ridge(), ridge_param_grid, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
ridge_grid.fit(X_train, y_train)

best_ridge_alpha = ridge_grid.best_params_['alpha']
print("Best Ridge Parameters:",best_ridge_alpha)

# PREDICTING WITH THE BEST MODEL
y_pred_ridge_best = ridge_grid.best_estimator_.predict(X_val)
mae_ridge_best = mean_absolute_error(y_val, y_pred_ridge_best)
print("Ridge Mean Absolute Error with Best Parameters:", mae_ridge_best)


Best Ridge Parameters: 1
Ridge Mean Absolute Error with Best Parameters: 1.4558299047313825


In [ ]:
#LASSO
lasso_param_grid = {'alpha': [0.001, 0.01, 0.1, 1, 10, 100]}
lasso_grid = GridSearchCV(Lasso(), lasso_param_grid, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
lasso_grid.fit(X_train, y_train)

best_lasso_alpha = lasso_grid.best_params_['alpha']
print("Best Lasso Parameters:",best_lasso_alpha)

# PREDICTING WITH THE BEST MODEL
y_pred_lasso_best = lasso_grid.best_estimator_.predict(X_val)
mae_lasso_best = mean_absolute_error(y_val, y_pred_lasso_best)
print("Lasso Mean Absolute Error with Best Parameters:", mae_lasso_best)

Best Lasso Parameters: 0.001
Lasso Mean Absolute Error with Best Parameters: 1.4555239184244892


In [ ]:
#ELASTIC NET
elastic_net_param_grid = {'alpha': [0.001, 0.01, 0.1, 1, 10, 100],
                           'l1_ratio': [0.1, 0.5, 0.9, 1.0]}
elastic_net_grid = GridSearchCV(ElasticNet(), elastic_net_param_grid, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
elastic_net_grid.fit(X_train, y_train)

best_elastic_net_alpha = elastic_net_grid.best_params_['alpha']
best_elastic_net_l1_ratio = elastic_net_grid.best_params_['l1_ratio']

print("Best Elastic Net Parameters:", elastic_net_grid.best_params_)

# PREDICTING WITH THE BEST MODEL
y_pred_en_best = elastic_net_grid.best_estimator_.predict(X_val)
mae_en_best = mean_absolute_error(y_val, y_pred_en_best)
print("Elastic Net Mean Absolute Error with Best Parameters:", mae_en_best)

Best Elastic Net Parameters: {'alpha': 0.001, 'l1_ratio': 1.0}
Elastic Net Mean Absolute Error with Best Parameters: 1.4555239184244892


WE WILL NOW TRY USING POLYNOMIAL REGRESSION

In [ ]:
degrees = range(1, 5)

In [ ]:
#POLYNOMIAL REGRESSION
mae_list = []

for degree in degrees:

    poly = PolynomialFeatures(degree=degree)
    X_train_poly = poly.fit_transform(X_train)
    X_val_poly = poly.transform(X_val)

    poly_reg_model = LinearRegression()
    poly_reg_model.fit(X_train_poly, y_train)

    y_pred_poly = poly_reg_model.predict(X_val_poly)

    mae_poly = mean_absolute_error(y_val, y_pred_poly)
    mae_list.append(mae_poly)
    print(f"Degree: {degree}, MAE: {mae_poly}")

best_degree = degrees[np.argmin(mae_list)]
best_mae = min(mae_list)

print(f"Best degree: {best_degree}, with MAE: {best_mae}")




Degree: 1, MAE: 1.4561177082682946
Degree: 2, MAE: 1.4114037713522327
Degree: 3, MAE: 1.405828786239214
Degree: 4, MAE: 1.4624792258331187
Degree: 5, MAE: 1.69641458374379
Degree: 6, MAE: 4.2934810104434415


Now we will try hybrid methods including both Polynomial Regression as well as Regularization Techniques


In [ ]:
#POLY - RIDGE
poly_ridge_mae_list = []
for degree in degrees:

      ridge_poly_model = make_pipeline(PolynomialFeatures(degree=degree),
                                      Ridge(alpha=best_ridge_alpha))
      ridge_poly_model.fit(X_train, y_train)

      y_pred_ridge_poly = ridge_poly_model.predict(X_val)
      mae_ridge_poly = mean_absolute_error(y_val, y_pred_ridge_poly)
      poly_ridge_mae_list.append(mae_ridge_poly)

      print(f"Degree: {degree}, MAE: {mae_ridge_poly}")

best_degree = degrees[np.argmin(poly_ridge_mae_list)]
best_mae = min(poly_ridge_mae_list)
print(f"Best degree: {best_degree}, with MAE: {best_mae}")

Degree: 1, MAE: 1.455829904731397
Degree: 2, MAE: 1.4107694604028957
Degree: 3, MAE: 1.3974639855018474
Degree: 4, MAE: 1.4251190816204435
Best degree: 3, with MAE: 1.3974639855018474


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=4.03426e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


In [ ]:
#POLY - LASSO
poly_lasso_mae_list = []
for degree in degrees:

      lasso_poly_model = make_pipeline(PolynomialFeatures(degree=degree),
                                      Lasso(alpha=best_lasso_alpha))
      lasso_poly_model.fit(X_train, y_train)

      y_pred_lasso_poly = lasso_poly_model.predict(X_val)
      mae_lasso_poly = mean_absolute_error(y_val, y_pred_lasso_poly)
      poly_lasso_mae_list.append(mae_lasso_poly)

      print(f"Degree: {degree}, MAE: {mae_lasso_poly}")

best_degree = degrees[np.argmin(poly_lasso_mae_list)]
best_mae = min(poly_lasso_mae_list)
print(f"Best degree: {best_degree}, with MAE: {best_mae}")


Degree: 1, MAE: 1.4555239184244892


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.384e+04, tolerance: 1.231e+01
  model = cd_fast.enet_coordinate_descent(


Degree: 2, MAE: 1.4129223593364801


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.366e+04, tolerance: 1.231e+01
  model = cd_fast.enet_coordinate_descent(


Degree: 3, MAE: 1.4089604894581536
Degree: 4, MAE: 1.4075947657274015
Best degree: 4, with MAE: 1.4075947657274015


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.343e+04, tolerance: 1.231e+01
  model = cd_fast.enet_coordinate_descent(


In [ ]:
#POLY - ELASTIC NET
poly_en_mae_list = []
for degree in degrees:

      elastic_net_poly_model = make_pipeline(PolynomialFeatures(degree=degree),
                                      ElasticNet(alpha=best_elastic_net_alpha, l1_ratio=best_elastic_net_l1_ratio))
      elastic_net_poly_model.fit(X_train, y_train)

      y_pred_en_poly = elastic_net_poly_model.predict(X_val)
      mae_en_poly = mean_absolute_error(y_val, y_pred_en_poly)
      poly_en_mae_list.append(mae_en_poly)

      print(f"Degree: {degree}, MAE: {mae_en_poly}")

best_degree = degrees[np.argmin(poly_en_mae_list)]
best_mae = min(poly_en_mae_list)
print(f"Best degree: {best_degree}, with MAE: {best_mae}")

Degree: 1, MAE: 1.4555239184244892


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.384e+04, tolerance: 1.231e+01
  model = cd_fast.enet_coordinate_descent(


Degree: 2, MAE: 1.4129223593364801


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.366e+04, tolerance: 1.231e+01
  model = cd_fast.enet_coordinate_descent(


Degree: 3, MAE: 1.4089604894581536
Degree: 4, MAE: 1.4075947657274015
Best degree: 4, with MAE: 1.4075947657274015


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.343e+04, tolerance: 1.231e+01
  model = cd_fast.enet_coordinate_descent(


In [ ]:
#FUNCTION TO CREATE RESIDUAL PLOTS
def plot_residuals(y_true, y_pred, model_name):
    residuals = y_true - y_pred
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x=y_pred, y=residuals, alpha=0.5)
    plt.axhline(0, color='red', linestyle='--', lw=2)
    plt.title(f'Residual Plot for {model_name}')
    plt.xlabel('Predicted Values')
    plt.ylabel('Residuals')
    plt.grid()
    plt.show()

plot_residuals(y_val, y_pred_lasso_best, model_name='Lasso')

plot_residuals(y_val, y_pred_ridge_best, model_name='Ridge')

plot_residuals(y_val, y_pred_en_best, model_name='Elastic Net')


In [ ]:
#VISUALIZING THE MODEL
plt.figure(figsize=(10, 10))
plt.scatter(y_val, y_pred, alpha=1)
plt.plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()], 'k--', lw=2)
plt.xlabel('Actual Age')
plt.ylabel('Predicted Age')
plt.title('Actual vs. Predicted Age')
plt.show()

End of PCA(Train)

TESTINGGGGGGGGGGGGG
